In [1]:
from os import environ
os.environ["JPY_API_TOKEN"] = ""
os.environ["JUPYTERPOST_URL"] = "https://services.io.quantumtinkerer.group/services/jupyterpost"
import jupyterpost

In [2]:
from itertools import count

import sympy
import numpy as np
import tinyarray as ta

from polynomial_orders_U import compute_next_orders, H_tilde
from sympy import MatrixSymbol

In [3]:
wanted_orders = [[4]]
Ns = [2, 2] # np.random.randint(1, high=5, size=2)


def hamiltonians(Ns, wanted_orders):
    """
    Produce random Hamiltonians to test.

    Ns: dimension of each block (A, B)
    wanted_orders: list of orders to compute

    Returns:
    hams: list of Hamiltonians
    """
    N_p = len(wanted_orders[0])
    orders = ta.array(np.eye(N_p))
    hams = []
    for i in range(2):
        hams.append(np.diag(np.sort(np.random.rand(Ns[i])) - i))

    def matrices_it(N_i, N_j, hermitian):
        """
        Generate random matrices of size N_i x N_j.

        N_i: number of rows
        N_j: number of columns
        hermitian: if True, the matrix is hermitian

        Returns:
        generator of random matrices
        """
        for i in count():
            H = np.random.rand(N_i, N_j) + 1j * np.random.rand(N_i, N_j)
            if hermitian:
                H += H.conj().T
            yield H

    for i, j, hermitian in zip([0, 1, 0], [0, 1, 1], [True, True, False]):
        matrices = matrices_it(Ns[i], Ns[j], hermitian)
        hams.append({order: matrix for order, matrix in zip(orders, matrices)})
    return hams



In [4]:
hams = hamiltonians(Ns, wanted_orders)
H_0_AA, H_0_BB, H_p_AA, H_p_BB, H_p_AB = hams
exp_S = compute_next_orders(*hams, wanted_orders=wanted_orders)

H_AA_01 = H_tilde(H_0_AA, np.zeros_like(H_0_BB), {}, {}, {}, wanted_orders, exp_S)[0]
H_AA_02 = H_tilde(H_0_AA, H_0_BB, {}, {}, {}, wanted_orders, exp_S)[0]
H_AA_p1 = H_tilde(np.zeros_like(H_0_AA), np.zeros_like(H_0_BB), H_p_AA, {}, {}, wanted_orders, exp_S)[0]
H_AA_p2 = H_tilde(np.zeros_like(H_0_AA), np.zeros_like(H_0_BB), {}, H_p_BB, {}, wanted_orders, exp_S)[0]
H_AA_p3 = H_tilde(np.zeros_like(H_0_AA), np.zeros_like(H_0_BB), {}, {}, H_p_AB, wanted_orders, exp_S)[0]

In [6]:
# for order in H_AA_0:
#     print(H_AA_0[order] / H_AA_p[order])
#     print()
#     print()


In [7]:
# test
from sympy.physics.quantum import Operator, HermitianOperator
from operator import mul

dim1 = 2
dim2 = 2

H_0_AA = HermitianOperator("{H_{0}^{AA}}")
H_0_BB = HermitianOperator("{H_{0}^{BB}}")

H_p_AA = {}
H_p_BB = {}
H_p_AB = {}

H_p_AA[ta.array([1])] = HermitianOperator("{H_{p}^{AA}}")
H_p_BB[ta.array([1])] = HermitianOperator("{H_{p}^{BB}}")   
H_p_AB[ta.array([1])] = Operator("{H_{p}^{AB}}")

In [8]:
class EnergyDivider:
    def __init__(self):
        self.data = {}

    def __call__(self, rhs):
        new_entry = Operator(f"V_{len(self.data) + 1}")
        self.data[new_entry] = rhs
        return new_entry

In [9]:
wanted_orders=[[5]]
divider = EnergyDivider()
exp_S = compute_next_orders(
    H_0_AA, H_0_BB, H_p_AA, H_p_BB, H_p_AB,
    wanted_orders=wanted_orders,
    divide_energies=divider,
    op=mul,
)


In [53]:
H_tilde_AA, H_tilde_BB = H_tilde(
    H_0_AA, H_0_BB, H_p_AA, H_p_BB, H_p_AB, wanted_orders, exp_S, op=mul
)


In [55]:
H_tilde_AA[(2,)]

-{H_{0}^{AA}}*V_1*Dagger(V_1)/2 - V_1*Dagger(V_1)*{H_{0}^{AA}}/2 - V_1*Dagger({H_{p}^{AB}}) + V_1*{H_{0}^{BB}}*Dagger(V_1) - {H_{p}^{AB}}*Dagger(V_1)

In [57]:
H_tilde_AA[(4,)]

(-V_1*Dagger(V_2) - V_2*Dagger(V_1))*{H_{p}^{AA}}/2 - (-V_1*Dagger({H_{p}^{AB}})*V_1*Dagger(V_1)/2 + V_3*Dagger({H_{p}^{AB}})) + (-V_1*Dagger(V_1)*V_1*Dagger(V_1)/4 - V_1*Dagger(V_3) - V_2*Dagger(V_2) - V_3*Dagger(V_1))*{H_{0}^{AA}}/2 - (-V_1*{H_{0}^{BB}}*Dagger(V_3) - V_1*{H_{p}^{BB}}*Dagger(V_2) - V_2*{H_{0}^{BB}}*Dagger(V_2) - V_2*{H_{p}^{BB}}*Dagger(V_1) - V_3*{H_{0}^{BB}}*Dagger(V_1)) + {H_{0}^{AA}}*(-V_1*Dagger(V_1)*V_1*Dagger(V_1)/4 - V_1*Dagger(V_3) - V_2*Dagger(V_2) - V_3*Dagger(V_1))/2 + {H_{p}^{AA}}*(-V_1*Dagger(V_2) - V_2*Dagger(V_1))/2 + V_1*Dagger(V_1)*{H_{0}^{AA}}*V_1*Dagger(V_1)/4 + V_1*Dagger(V_1)*{H_{p}^{AB}}*Dagger(V_1)/2 - {H_{p}^{AB}}*Dagger(V_3)

In [74]:
sympy.simplify(list(divider.data.values())[-1])

(V_1*Dagger(V_2) + V_2*Dagger(V_1))*{H_{0}^{AA}}*V_2/2 + (V_1*Dagger(V_2) + V_2*Dagger(V_1))*{H_{p}^{AA}}*V_1/2 + ((V_1*Dagger(V_1))**2 + 4*V_1*Dagger(V_3) + 4*V_2*Dagger(V_2) + 4*V_3*Dagger(V_1))*{H_{0}^{AA}}*V_1/8 + ((V_1*Dagger(V_1))**2 + 4*V_1*Dagger(V_3) + 4*V_2*Dagger(V_2) + 4*V_3*Dagger(V_1))*{H_{p}^{AB}}/8 - {H_{p}^{AA}}*V_4 + V_1*Dagger(V_1)*{H_{0}^{AA}}*V_3/2 + V_1*Dagger(V_1)*{H_{p}^{AA}}*V_2/2 - V_1*Dagger(V_1)*{H_{p}^{AB}}*Dagger(V_1)*V_1/4 + V_1*Dagger({H_{p}^{AB}})*V_3 - V_1*{H_{0}^{BB}}*((Dagger(V_1)*V_1)**2 + 4*Dagger(V_1)*V_3 + 4*Dagger(V_2)*V_2 + 4*Dagger(V_3)*V_1)/8 - V_1*{H_{p}^{BB}}*(Dagger(V_1)*V_2 + Dagger(V_2)*V_1)/2 + V_2*Dagger({H_{p}^{AB}})*V_2 - V_2*{H_{0}^{BB}}*(Dagger(V_1)*V_2 + Dagger(V_2)*V_1)/2 - V_2*{H_{p}^{BB}}*Dagger(V_1)*V_1/2 + V_3*Dagger({H_{p}^{AB}})*V_1 - V_3*{H_{0}^{BB}}*Dagger(V_1)*V_1/2 + V_4*{H_{p}^{BB}} + {H_{p}^{AB}}*((Dagger(V_1)*V_1)**2 + 4*Dagger(V_1)*V_3 + 4*Dagger(V_2)*V_2 + 4*Dagger(V_3)*V_1)/8

In [89]:
expr = sympy.expand(list(divider.data.values())[-2])

In [91]:
list(divider.data.values())[1]

-{H_{p}^{AA}}*V_1 + V_1*{H_{p}^{BB}}

In [90]:
sympy.expand(expr.subs({H_0_AA * v: v * H_0_BB + rhs for v, rhs in divider.data.items()}).subs({H_0_AA * v: v * H_0_BB + rhs for v, rhs in divider.data.items()}))

-{H_{p}^{AA}}*V_3 + V_1*Dagger(V_1)*V_1*{H_{p}^{BB}}/2 + V_1*Dagger(V_1)*V_2*{H_{0}^{BB}}/2 + V_1*Dagger(V_2)*V_1*{H_{0}^{BB}}/2 + V_1*Dagger({H_{p}^{AB}})*V_2 - V_1*{H_{0}^{BB}}*Dagger(V_1)*V_2/2 - V_1*{H_{0}^{BB}}*Dagger(V_2)*V_1/2 - V_1*{H_{p}^{BB}}*Dagger(V_1)*V_1/2 + V_2*Dagger(V_1)*V_1*{H_{0}^{BB}}/2 + V_2*Dagger({H_{p}^{AB}})*V_1 - V_2*{H_{0}^{BB}}*Dagger(V_1)*V_1/2 + V_3*{H_{p}^{BB}} + {H_{p}^{AB}}*Dagger(V_1)*V_2/2 + {H_{p}^{AB}}*Dagger(V_2)*V_1/2

In [60]:
H_tilde_AA[(4,)]

(-V_1*Dagger(V_2) - V_2*Dagger(V_1))*{H_{p}^{AA}}/2 - (-V_1*Dagger({H_{p}^{AB}})*V_1*Dagger(V_1)/2 + V_3*Dagger({H_{p}^{AB}})) + (-V_1*Dagger(V_1)*V_1*Dagger(V_1)/4 - V_1*Dagger(V_3) - V_2*Dagger(V_2) - V_3*Dagger(V_1))*{H_{0}^{AA}}/2 - (-V_1*{H_{0}^{BB}}*Dagger(V_3) - V_1*{H_{p}^{BB}}*Dagger(V_2) - V_2*{H_{0}^{BB}}*Dagger(V_2) - V_2*{H_{p}^{BB}}*Dagger(V_1) - V_3*{H_{0}^{BB}}*Dagger(V_1)) + {H_{0}^{AA}}*(-V_1*Dagger(V_1)*V_1*Dagger(V_1)/4 - V_1*Dagger(V_3) - V_2*Dagger(V_2) - V_3*Dagger(V_1))/2 + {H_{p}^{AA}}*(-V_1*Dagger(V_2) - V_2*Dagger(V_1))/2 + V_1*Dagger(V_1)*{H_{0}^{AA}}*V_1*Dagger(V_1)/4 + V_1*Dagger(V_1)*{H_{p}^{AB}}*Dagger(V_1)/2 - {H_{p}^{AB}}*Dagger(V_3)